# 단일 입력 프롬프트 템플릿 

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
import langchain
langchain.__version__

'0.3.25'

In [16]:
# 탬플릿 문자열 : 프롬프트의 골격을 정의
# promptTemplate 객체 생성 : 인스턴스 변수 선언
# 포멧(format) 메서드 수행 : promptTemplate.format() 
#LLM 호출 
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다. \n"
    "아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오.\n\n"
    "제품 명:{product_name}"
)

prompt = PromptTemplate.from_template(template_str)

input_str = input("제품명을 입력하세여 : ")
formatted_prompt = prompt.format(product_name =input_str)
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다. 
아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오.

제품 명:초 슈퍼


In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4.1-nano",temperature=0)

chain = prompt | llm
result = chain.invoke({"product_name": input_str})
print(result.content)

초 슈퍼와 함께라면 일상이 더 활기차지고 건강해집니다! 자연의 힘으로 가득 찬 최고의 선택, 지금 경험하세요!


# 다중 입력 PromptTemplate

In [22]:
multi_tempalte_str = (
    "아래는 뉴스 기사 제목과 키워드입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오. \n\n"
    "제목 : {title}\n"
    "키워드 : {keywords}\n"
)

summary_prompt = PromptTemplate(template=multi_tempalte_str,input_variables=["title","keywords"])
sample_title = "인공지능 발전과 미래"
sample_key = "머신러닝,딥러닝,산업,혁신,llm"
formatted_summary_prompt = summary_prompt.format(title =sample_title,keywords = sample_key)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오. 

제목 : 인공지능 발전과 미래
키워드 : 머신러닝,딥러닝,산업,혁신,llm



In [24]:
result_summary = (summary_prompt | llm).invoke(
    {
        "title":sample_title,
        "keywords":sample_key
    }
)

print(result_summary.content)

인공지능의 발전은 머신러닝과 딥러닝 기술의 발전을 바탕으로 산업 전반에 혁신을 가져오고 있으며, 특히 대규모 언어 모델(LLM)의 등장으로 인공지능의 활용 범위와 성능이 크게 향상되고 있어 미래 산업의 변화와 혁신이 기대되고 있습니다.


# ChatPromptTemplate

-   역할 기반 프롬프트

In [25]:
from langchain.prompts import ChatPromptTemplate


In [29]:
# 시나리오 : 파이썬 프로그램 전문가 역할 부여 , 사용자가 파이썬 관련 질문에 친절히, 간결한 답변 하도록
sys_message = ("당신은 파이썬 분야의 뛰어난 전문가입니다."
               "사용자의 프로그래밍 질문에 친절하고 이해하기 쉽게 답변하십시오.")
user_messaqe = "{question}" # 사용자 질문 -> 실행 시 입력값으로 채워진다.

chat_prompt = ChatPromptTemplate.from_messages([
    ("system",sys_message),
    ("user",user_messaqe)
])

sample_question = "파이썬의 리스트 컴프리헨션(list comprehension)과 map함수의 차이가 무엇이야?"

message = chat_prompt.format_messages(question = sample_question)
print(message)

[SystemMessage(content='당신은 파이썬 분야의 뛰어난 전문가입니다.사용자의 프로그래밍 질문에 친절하고 이해하기 쉽게 답변하십시오.', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 리스트 컴프리헨션(list comprehension)과 map함수의 차이가 무엇이야?', additional_kwargs={}, response_metadata={})]


In [30]:
result = (chat_prompt | llm ).invoke({
    "question" : sample_question
})

print(result.content)

안녕하세요! 파이썬에서 리스트 컴프리헨션과 `map()` 함수는 둘 다 기존 리스트(또는 이터러블)의 각 요소에 대해 어떤 작업을 수행해서 새 리스트를 만드는 데 사용되지만, 몇 가지 차이점이 있어요. 아래에서 쉽게 설명해 드릴게요.

---

### 1. 문법과 가독성
- **리스트 컴프리헨션**은 더 직관적이고 간결한 문법을 가지고 있어요.
  
  ```python
  # 예: 0부터 4까지의 제곱
  squares = [x**2 for x in range(5)]
  ```

- **`map()` 함수**는 함수와 이터러블을 인자로 받아서, 그 함수가 이터러블의 각 요소에 적용된 결과를 반환해요.
  
  ```python
  def square(x):
      return x**2

  squares = list(map(square, range(5)))
  ```

---

### 2. 함수 사용 방식
- **리스트 컴프리헨션**은 내부에 조건문이나 복잡한 표현식을 넣을 수 있어요.
  
  ```python
  # 짝수만 제곱
  even_squares = [x**2 for x in range(10) if x % 2 == 0]
  ```

- **`map()`**은 보통 하나의 함수만 적용할 수 있고, 조건문을 넣으려면 별도로 필터링이 필요해요.
  
  ```python
  def is_even(x):
      return x % 2 == 0

  even_numbers = list(filter(is_even, range(10)))
  ```

---

### 3. 성능
- 일반적으로 **리스트 컴프리헨션**이 약간 더 빠른 경우가 많아요. 하지만 차이는 미미해서, 가독성과 편의성을 우선시하는 게 좋아요.

---

### 4. 유연성
- **리스트 컴프리헨션**은 복잡한 조건이나 여러 표현식을 넣기 쉽고, 가독성도 좋아서 복잡한 작업에 적합해요.
- **`map()`**은 간단한 함수 적용에 적합하며, 이미 정의된 함수를 재사용할 때 유리해요.

# PartialPromptTemplate

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

